You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re

### Importo i dataset

Per importare inserisco tra i parametri il compressore da utilizzare, l'encoding, il separatore e dove necessario l'escape character. Inoltre dove necessario specifico il tipo di ogni attributo cosí da notare eventuali problemi con qualche riga dei dataframe.

Per il dataset goodbooks sarebbe meglio usare l'encoding utf-8.

In [3]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'int',
                                                                                                                                                                                    'Location': 'string',
                                                                                                                                                                                    'Age': 'Int64'
                                                                                                                                                                                    })
users.shape

(278858, 3)

In [51]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'ISBN': 'string',
                                                                                                                                                                                  'Book-title': 'string',
                                                                                                                                                                                  'Book-Author': 'string',
                                                                                                                                                                                  'Year-Of-Publication': 'int',
                                                                                                                                                                                  'Publisher': 'string',
                                                                                                                                                                                  'Image-URL-S': 'string',
                                                                                                                                                                                  'Image-URL-M': 'string',
                                                                                                                                                                                  'Image-URL-L': 'string',
                                                                                                                                                                                 })
books.shape

(271359, 8)

In [5]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'User-ID': 'Int64',
                                                                                                                                                                                 'ISBN': 'string',
                                                                                                                                                                                 'Book-Rating': 'Int64'
                                                                                                                                                                                })
book_ratings['ISBN'] = book_ratings['ISBN'].str.strip(' "')
book_ratings['ISBN'] = [re.sub('[\W+]', '', s) for s in book_ratings['ISBN']] #escludere i punti di domanda da regex
book_ratings.shape

(1149780, 3)

In [6]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'user_id': 'int',
                                                                                                                                                                                        'book_id': 'int',
                                                                                                                                                                                        'rating': 'int'
                                                                                                                                                                                        })
goodbooks_ratings.shape

(99, 3)

In [7]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',', dtype = {'book_id': 'int',
                                                                                                                                                                            'goodreads_book_id': 'int',
                                                                                                                                                                            'best_book_id': 'int',
                                                                                                                                                                            'work_id': 'int',
                                                                                                                                                                            'books_count': 'int',
                                                                                                                                                                            'isbn': 'string',
                                                                                                                                                                            'isbn13': 'string',
                                                                                                                                                                            'authors': 'string',
                                                                                                                                                                            'original_publication_year': 'int',
                                                                                                                                                                            'original_title': 'string',
                                                                                                                                                                            'title': 'string',
                                                                                                                                                                            'language_code': 'string',
                                                                                                                                                                            'average_rating': 'double',
                                                                                                                                                                            'ratings_count': 'int',
                                                                                                                                                                            'work_ratings_count': 'int',
                                                                                                                                                                            'work_text_reviews_count': 'int',
                                                                                                                                                                            'ratings_1': 'int',
                                                                                                                                                                            'ratings_2': 'int',
                                                                                                                                                                            'ratings_3': 'int',
                                                                                                                                                                            'ratings_4': 'int',
                                                                                                                                                                            'ratings_5': 'int',
                                                                                                                                                                            'image_url': 'string',
                                                                                                                                                                            'small_image_url': 'string'
                                                                                                                                                                         })
goodbooks.shape

(99, 23)

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [64]:
users[['City', 'Region', 'Country']] = users['Location'].str.rsplit(',', 2, expand=True)
users.head()

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa
1,2,"stockton, california, usa",18,stockton,california,usa
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom


Il right split permette di dividere le stringhe a partire dal valore Country per tentare di risolvere alcuni problemi con le righe con piú di due virgole nel campo Location.
Facendo un group by rispetto a Country ci si rende conto della presenza di valori anomali in alcuni campi, inoltre si notano alcuni problemi con l'encoding.

### 2. For each book in the *Books* dataset, compute its average rating.

In [65]:
pd.merge(books, book_ratings, on = 'ISBN', how = 'left').groupby(['ISBN', 'Book-Title'], as_index=False).agg(Average_Rating=('Book-Rating', 'mean')).head()

,ISBN,Book-Title,Average_Rating
0,0000913154,The Way Things Work: An Illustrated Encycloped...,8.0
1,0001010565,Mog's Christmas,0.0
2,0001046438,Liar,9.0
3,0001046713,Twopence to Cross the Mersey,0.0
4,000104687X,"T.S. Eliot Reading ""The Wasteland"" and Other P...",6.0


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [66]:
goodbooks[['isbn', 'original_title', 'average_rating']].head()

,isbn,original_title,average_rating
0,439023483,The Hunger Games,4.34
1,439554934,Harry Potter and the Philosopher's Stone,4.44
2,316015849,Twilight,3.57
3,61120081,To Kill a Mockingbird,4.25
4,743273567,The Great Gatsby,3.89


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [67]:
merging_books = books[books.duplicated(subset = ['Book-Title', 'Book-Author', 'Publisher'], keep = False)].sort_values(['Book-Title', 'Book-Author', 'Publisher'])
merged_books = merging_books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg(Count=('ISBN', 'count'))
merged_books.head()

,Book-Title,Book-Author,Publisher,Count
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


Ho inizialmente creato un dataset merging_books, che sfrutteró anche nel punto successivo, dove con la funzione duplicated con keep = False tengo solo le righe che hanno i campi [Book-Title, Book-Author, Publisher] uguali.

Successivamente creo il dataset merged_books facendo un group by del dataset precedente rispetto a [Book-Title, Book-Author, Publisher].

### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [63]:
book_ratings_avg = book_ratings.groupby('ISBN', as_index=True).agg(Average_Rating = ('Book-Rating', 'mean'))
merging_book_ratings = pd.merge(merging_books, book_ratings_avg, on = 'ISBN', how = 'left')
merging_book_ratings.head() [['ISBN', 'Book-Title', 'Average_Rating']]

,ISBN,Book-Title,Average_Rating
0,1565920465,!%@ (A Nutshell handbook),0.0
1,1565920317,!%@ (A Nutshell handbook),6.0
2,0792276833,'A Hell of a Place to Lose a Cow': An American...,0.0
3,0792277295,'A Hell of a Place to Lose a Cow': An American...,3.4
4,1566196256,"10,000 dreams interpreted: A dictionary of dreams",7.25


Calcolo l'Average_Rating per ogni libro in book_ratings e poi faccio un merge con il dataset creato al punto precedente.

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [13]:
merging_book_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg({'Average_Rating' : ['min', 'max']}).head()

Book-Title              Book-Author  \
                                                                                
0                          !%@ (A Nutshell handbook)            Donnalyn Frey   
1  'A Hell of a Place to Lose a Cow': An American...              Tim Brookes   
2  10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller   
3                                   101 Famous Poems              Roy J. Cook   
4                 15 Houseplants Even You Can't Kill                Joe Elder   

                        Publisher Average_Rating        
                                             min   max  
0                        O'Reilly            0.0   6.0  
1             National Geographic            0.0   3.4  
2       Barnes &amp; Nobles Books       6.666667  7.25  
3  McGraw-Hill/Contemporary Books            0.0   5.0  
4               Berkley Pub Group            0.0   0.0

Sfruttando il dataset del punto precedente facciamo un groupby e calcoliamo max e min del valore medio con la funzione agg

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [62]:
goodbooks['authors_splitted'] = goodbooks['authors'].str.split(', ')
goodbooks['shared_number_of_reviews_with_a_text'] = goodbooks['work_text_reviews_count'] / goodbooks['authors_splitted'].apply(len)

goodbooks_exploded = goodbooks.explode('authors_splitted')
goodbooks_exploded.groupby(['authors_splitted'], as_index=False)['shared_number_of_reviews_with_a_text'].sum().head()

,authors_splitted,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,Aldous Huxley,20095.000000
2,Alice Sebold,36642.000000
3,Anne Frank,6941.666667
4,Antoine de Saint-ExupÃ©ry,6134.250000


Creiamo una nuova colonna chiamata authors_splitted, ottenuta con la funzione split, che contiene gli autori come lista per ogni libro.
Aggiungiamo poi la colonna shared_number_of_reviews_with_a_text calcolata dividendo work_text_reviews_count per in numero di autori di ogni libro (lunghezza della lista splitted_authors).

Costruiamo poi un nuovo dataset con una riga per ogni coppia libro-autore della lista splitted_authors. Infine facciamo un group by e una sum rispetto ad ogni attore per ottenere il shared_number_of_reviews_with_a_text di ogni scrittore.

### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [15]:
temp = goodbooks_exploded.groupby(['original_publication_year', 'authors_splitted'], as_index=False).sum()
idx = temp.groupby(['original_publication_year'])['shared_number_of_reviews_with_a_text'].transform(max) == temp['shared_number_of_reviews_with_a_text']

authors_highest_reviews = temp[idx] [['original_publication_year', 'shared_number_of_reviews_with_a_text', 'authors_splitted']]
authors_highest_reviews.set_index(['original_publication_year', 'shared_number_of_reviews_with_a_text', 'authors_splitted']).head(10)

Empty DataFrame
Columns: []
Index: [(-720, 1620.2, Bernard Knox), (-720, 1620.2, E.V. Rieu), (-720, 1620.2, FrÃ©dÃ©ric Mugler), (-720, 1620.2, Homer), (-720, 1620.2, Robert Fagles), (1595, 7389.0, Robert           Jackson), (1595, 7389.0, William Shakespeare), (1811, 3842.0, Jane Austen), (1811, 3842.0, Ros Ballaster), (1811, 3842.0, Tony Tanner)]

Da spiegare bene..

Infine creo un altro dataset temporaneo chiamato authors_highest_reviews cosí da poter usare set_index per avere una vista piu compatta per gli anni con autori a pari merito.

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [16]:
books_goodbooks = pd.merge(books, goodbooks, left_on = 'ISBN', right_on = 'isbn')
books_goodbooks = pd.merge(books_goodbooks, book_ratings, on = 'ISBN', how = 'left').groupby('ISBN', as_index=False).mean()

books_goodbooks['Book-Rating'] = books_goodbooks['Book-Rating'] / 2#scala 5 dei goodbooks e 10 dei books
books_goodbooks['Difference'] = books_goodbooks['Book-Rating'] - books_goodbooks['average_rating']

books_goodbooks.rename(columns={'Book-Rating': 'Book_Average_Rating', 'average_rating': 'Goodbook_Average_Rating'}, inplace=True)
books_goodbooks[['ISBN','Book_Average_Rating', 'Goodbook_Average_Rating', 'Difference']]

,ISBN,Book_Average_Rating,Goodbook_Average_Rating,Difference
0,014028009X,1.888278,3.75,-1.861722
1,014038572X,2.244565,4.06,-1.815435
2,043965548X,1.766667,4.53,-2.763333
3,1400032717,2.406593,3.85,-1.443407
4,1594480001,1.2,4.26,-3.06


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [69]:
users.groupby(pd.cut(users['Age'], np.arange(14, 150, 10)), as_index=False).count()

,User-ID,Location,Age,City,Region,Country
0,40001,40001,40001,40001,40001,40001
1,50767,50767,50767,50767,50766,50766
2,32690,32690,32690,32690,32690,32690
3,23152,23152,23152,23152,23152,23152
4,12493,12493,12493,12493,12493,12493
5,3596,3596,3596,3596,3596,3595
6,615,615,615,615,615,615
7,83,83,83,83,83,83
8,278,278,278,278,278,278
9,20,20,20,20,20,20


In [70]:
interval = np.arange(14,95,10)
grouped_users=list(range(len(interval)))
grouped_users[0]=users[users["Age"]<=14][["User-ID","Age"]]
grouped_users[len(interval)-1]=users[users["Age"].isnull()][["User-ID","Age"]]
for i in range(1,len(interval)-1): #va da 1 a 7
    s=users[(users["Age"]>interval[i])&(users["Age"]<=interval[i+1])][["User-ID","Age"]]
    grouped_users[i]=s

grouped_users[2].head()

,User-ID,Age
53,54,44
66,67,43
69,70,44
74,75,37
84,85,41


### 11. Find the books that appear only in the goodbooks datasets.

In [71]:
goodbooks[~goodbooks['isbn'].isin(books_goodbooks['ISBN'])].head() [['isbn', 'original_title']]

,isbn,original_title
0,439023483,The Hunger Games
1,439554934,Harry Potter and the Philosopher's Stone
2,316015849,Twilight
3,61120081,To Kill a Mockingbird
4,743273567,The Great Gatsby


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [72]:
temp = books.groupby(['Book-Author', 'Book-Title'], as_index=False)['ISBN'].count()
temp.rename(columns={'ISBN':'Count'}, inplace=True)
temp.head()

,Book-Author,Book-Title,Count
0,D. Chiel,Die Hard: With a Vengeance - A Novel,1
1,J. D. Landis,The Taking,1
2,Mimma Balia,Ruskin's Rose: A Venetian Love Story,1
3,"""""Ron""",Lord Have Mercy,1
4,"""Abu""",Penguin Book of Indian Cartoons,1


In [73]:
temp[temp['Count'] == temp['Count'].max()]

,Book-Author,Book-Title,Count
150658,Louisa May Alcott,Little Women,21


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [74]:
temp = goodbooks_exploded.groupby('authors_splitted', as_index=False)['average_rating'].mean()
temp.loc[temp['average_rating'] == temp['average_rating'].max()]

,authors_splitted,average_rating
91,Rufus Beck,4.53
